# VAMOS API Guide: 3 Ways to Run Optimization

This notebook demonstrates the three primary ways to run optimization experiments in VAMOS, ranging from quick one-liners to fully reproducible production setups.

1. **Unified API (`vamos.optimize`)**: **Recommended**. The simplest way to run experiments, supporting AutoML and standard configurations.
2. **Advanced Config (explicit args)**: Best for full programmatic control, custom operators, and pipelines.
3. **Command Line (CLI)**: Best for reproducible batch processing, scaling, and automation.

If you do not need explicit operator control, stick with `optimize(...)`. For deeper guidance, see `docs/guide/getting-started.md`.


## 1. Unified API (`vamos.optimize`)

Use this for 90% of use cases. It intelligently configures standard algorithms and problems.

In [ ]:
import vamos

print("Running Unified API...")
# One-liner: Solve ZDT1 with NSGA-II for 5000 evaluations
result = vamos.optimize("zdt1", algorithm="nsgaii", max_evaluations=5000, seed=1, verbose=True)
print(f"Done! Found {len(result)} solutions.")
print(f"Objective ranges:")
print(f"  f1: [{result.F[:, 0].min():.4f}, {result.F[:, 0].max():.4f}]")
print(f"  f2: [{result.F[:, 1].min():.4f}, {result.F[:, 1].max():.4f}]")

## 2. Advanced Config (explicit args)

Use this when you need fine-grained control over operators, parameters, or when injecting custom problem objects.
Config objects and builders make those choices explicit and reproducible.


In [ ]:
from vamos import optimize
from vamos.algorithms import NSGAIIConfig
from vamos.problems import ZDT1

print("Running Advanced Config...")

# 1. Define Problem Instance Explicitly
problem = ZDT1(n_var=30)

# 2. Configure Algorithm Details
algo_config = (
    NSGAIIConfig.builder()
    .pop_size(100)
    # Precise operator control
    .crossover("sbx", prob=0.9, eta=15.0)
    .mutation("pm", prob="1/n", eta=20.0)
    .selection("tournament")
    
    .build()
)

# 3. Create explicit Config object
config = dict(problem=problem, algorithm="nsgaii", algorithm_config=algo_config, termination=("n_eval", 5000), seed=1)

# 4. Run
result = optimize(**config)
print(f"Advanced run finished with {len(result)} solutions.")

## 3. Configuration Files (YAML + CLI)

Use this for reproducible scaling, batch processing, and sharing experiment specifications.

If you are just getting started, try the guided wizard: `vamos quickstart`. It writes a config file
and stores results in `results/quickstart/`.

To see domain templates: `vamos quickstart --template list`.


In [ ]:
import yaml

spec = {
    "defaults": {
        "algorithm": "nsgaii",
        "problem": "zdt1",
        "max_evaluations": 5000,
        "seed": 1,
        "nsgaii": {"crossover": {"method": "sbx", "eta": 15.0}, "mutation": {"method": "pm", "prob": "1/n"}},
    }
}

with open("demo_spec.yaml", "w") as f:
    yaml.dump(spec, f)

print("Created demo_spec.yaml")

In [ ]:
!python -m vamos.experiment.cli.main --config demo_spec.yaml

In [ ]:
# Cleanup
import os

if os.path.exists("demo_spec.yaml"):
    os.remove("demo_spec.yaml")